In [2]:
import os
import pandas as pd
import panel as pn
import altair as alt
from toolz.curried import pipe

def json_dir(data, data_dir='altairdata'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )


alt.renderers.enable('default')
pn.extension('vega')

alt.data_transformers.register('json_dir', json_dir)
alt.data_transformers.enable('json_dir', data_dir='mydata')


DataTransformerRegistry.enable('json_dir')

In [3]:
# Lendo os dados do arquivo XLSX
file_path = '../../scrapper-dataset/excel/all-companies.xlsx'

data = pd.read_excel(file_path)

from tools.problemaAgrupador import realiza_agrupamento as ra1
from tools.produtoAgrupador import realiza_agrupamento as ra2
from tools.categoriaAgrupador import realiza_agrupamento as ra3

ra1(data)
ra2(data)
ra3(data)

In [15]:
data.head()

,id,titulo,status,data_criacao,reclamacao,estado,cidade,categoria,problema,produto,voltaria_fazer_negocio,nota,respondido_em,sla,problema_agrupado,produto_agrupado,categoria_agrupado
0,166933963,Bloqueio e suspensão do sinal sem aviso,Respondida,2023-06-22 19:46:00,Bloquearam meu sinal pq eu estava a 20 dias de...,SP,São Paulo,Provedores e serv. de internet,Mau atendimento no SAC,Internet para casa,False,NaN,2023-06-24 08:20:00,131640.0,Atendimento/Relacionamento,Serviços de Comunicação,Telecomunicações e Serviços
1,166928345,Falta de respeito e mentiras com o cliente,Não resolvido,2023-06-22 18:09:00,"boa tarde. executando uma ação judicial, pela ...",RJ,Duque de Caxias,Provedores e serv. de internet,Problema com equipamento fornecido,Internet para casa,False,0.0,2023-06-24 11:46:00,149820.0,Qualidade do Produto/Serviço,Serviços de Comunicação,Telecomunicações e Serviços
2,166918921,Assinatura Tim banda larga que dá acesso ao HB...,Resolvido,2023-06-22 15:57:00,"Contratei a TIM ultra fibra 500MG, com o pacot...",PR,Maringá,NaN,NaN,NaN,True,8.0,2023-06-24 13:32:00,164100.0,Outros,Outros,Outros
3,166864625,NUNCA CONTRATEI E ESTOU SENDO COBRADA !!!!,Em réplica,2023-06-21 20:00:00,URGENTE!!! Nunca contratei serviços de Banda L...,SP,São Paulo,Provedores e serv. de internet,Cobrança indevida,Internet para casa,False,NaN,2023-06-22 19:10:00,83400.0,Cobrança/Financeiro,Serviços de Comunicação,Telecomunicações e Serviços
4,166863225,TIM me liga várias vezes para me oferecer plano,Respondida,2023-06-21 19:37:00,A TIM me liga uma média de 3 vezes ao dia para...,RJ,Rio de Janeiro,Provedores e serv. de internet,Cancelamento,Internet para computador,False,NaN,2023-06-22 19:54:00,87420.0,Outros,Serviços de Comunicação,Telecomunicações e Serviços


In [6]:
top_10_categorias = data['categoria_agrupado'].value_counts().nlargest(10).index
top_10_problemas = data['problema_agrupado'].value_counts().nlargest(10).index
top_10_produtos = data['produto_agrupado'].value_counts().nlargest(10).index

# Filtrar os dados apenas para as top 10 categorias
filtered_categoria_df = data[data['categoria_agrupado'].isin(top_10_categorias)]
filtered_problema_df = data[data['problema_agrupado'].isin(top_10_problemas)]
filtered_produto_df = data[data['produto_agrupado'].isin(top_10_produtos)]

In [7]:
media_por_categoria = filtered_categoria_df.groupby('categoria_agrupado')['nota'].mean().reset_index()
media_por_problema = filtered_problema_df.groupby('problema_agrupado')['nota'].mean().reset_index()
media_por_produto = filtered_produto_df.groupby('produto_agrupado')['nota'].mean().reset_index()


In [8]:
df_media_resolucao_cat = data.groupby('categoria_agrupado')['sla'].mean().reset_index()
df_media_resolucao_cat = df_media_resolucao_cat.sort_values('sla', ascending=False)
df_top_10_cat = df_media_resolucao_cat.head(10)
df_top_10_cat['sla'] = df_top_10_cat['sla'] / 60 / 60 / 24 # Converter para dias

df_media_resolucao_prob = data.groupby('problema_agrupado')['sla'].mean().reset_index()
df_media_resolucao_prob = df_media_resolucao_prob.sort_values('sla', ascending=False)
df_top_10_prob = df_media_resolucao_prob.head(10)
df_top_10_prob['sla'] = df_top_10_prob['sla'] / 60 / 60 / 24 # Converter para dias

df_prob = df_media_resolucao_prob
df_prob['sla'] = df_prob['sla'] / 60 / 60 / 24 # Converter para dias

df_media_resolucao_prod = data.groupby('produto_agrupado')['sla'].mean().reset_index()
df_media_resolucao_prod = df_media_resolucao_prod.sort_values('sla', ascending=False)
df_top_10_prod = df_media_resolucao_prod.head(10)
df_top_10_prod['sla'] = df_top_10_prod['sla'] / 60 / 60 / 24 # Converter para dias

df_prod = df_media_resolucao_prod
df_prod['sla'] = df_prod['sla'] / 60 / 60 / 24 # Converter para dias

C:\Users\thiag\AppData\Local\Temp\ipykernel_3744\3570870626.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_10_prod['sla'] = df_top_10_prod['sla'] / 60 / 60 / 24 # Converter para dias


In [9]:
# Criando o gráfico de barras com Altair
chart_categoria = alt.Chart(filtered_categoria_df).mark_bar().encode(
    x=alt.X('count():Q', axis=None),
    y=alt.Y('categoria_agrupado:N', sort='-x', title='Categoria'),
    tooltip=['categoria_agrupado:N', 'count():Q']

).properties(
    title='Top 10 Categorias de Reclamações'
).interactive()

chart_problema = alt.Chart(filtered_problema_df).mark_bar().encode(
    x=alt.X('count():Q', axis=None),
    y=alt.Y('problema_agrupado:N', sort='-x', title='Problema'),
    tooltip=['problema_agrupado:N', 'count():Q']

).properties(
    title='Top 10 Problemas de Reclamações'
).interactive()

chart_produto = alt.Chart(filtered_produto_df).mark_bar().encode(
    x=alt.X('count():Q', axis=None),
    y=alt.Y('produto_agrupado:N', sort='-x', title='Produto'),
    tooltip=['produto_agrupado:N', 'count():Q']

).properties(
    title='Top 10 Produtos de Reclamações'
).interactive()


# Exibindo o gráfico
chart_categoria &chart_problema & chart_produto


alt.VConcatChart(...)

In [10]:
# Criando o gráfico de barras com Altair
chart_cat = alt.Chart(media_por_categoria).mark_bar().encode(
    y=alt.Y('categoria_agrupado:N', sort='-x'),
    x='nota'
)
chart_prob = alt.Chart(media_por_problema).mark_bar().encode(
    y=alt.Y('problema_agrupado:N', sort='-x'),
    x='nota'
)

chart_prod = alt.Chart(media_por_produto).mark_bar().encode(
    y=alt.Y('produto_agrupado:N', sort='-x'),
    x='nota'
)

# Exibindo o gráfico
chart_cat &chart_prob &chart_prod

alt.VConcatChart(...)

In [11]:
chart_cat = alt.Chart(df_top_10_cat).mark_bar().encode(
    y=alt.Y('categoria_agrupado:N', title='categoria', sort='-x'),
    x=alt.X('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('categoria_agrupado:N', legend=None),
    tooltip=['categoria_agrupado:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Categoria'
)

chart_prob = alt.Chart(df_top_10_prob).mark_bar().encode(
    y=alt.Y('problema_agrupado:N', title='problema', sort='-x'),
    x=alt.X('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('problema_agrupado:N', legend=None),
    tooltip=['problema_agrupado:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Problema'
)


chart_prod = alt.Chart(df_top_10_prod).mark_bar().encode(
    y=alt.Y('produto_agrupado:N', title='produto', sort='-x'),
    x=alt.X('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('produto_agrupado:N', legend=None),
    tooltip=['produto_agrupado:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Produto'
)
chart_cat & chart_prob & chart_prod

alt.VConcatChart(...)